In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2023-12-01'
final_date = '2023-12-31'
query_error_year = 2023
query_error_month = 12

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2023\12. DICIEMBRE\DATA\ERROR DICIEMBRE 2023.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_7608\1332134195.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2023\12. DICIEMBRE\DATA\ERROR DICIEMBRE 2023.csv',


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,nan
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,nan
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,nan
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,nan
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,nan


In [4]:
query_error.shape

(155949, 17)

In [5]:
query_error['mes_cierre'].unique()

array(['nan', 'Actual'], dtype=object)

In [6]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int32
year                             int32
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int32
mes_cierre                      object
dtype: object

In [7]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [8]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [9]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_7608\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
145090,145090,943839fe-a7a2-4ac9-96e3-51703832cdaa,NaN,APTLXZ4QH,2023-12-01,98.69,ERROR,/FLDVAL_1/Please enter the Beneficiary Name. //,2023-12-01,12,2023,NEW,ERROR_USUAL,943839fe-a7a2-4ac9-96e3-51703832cdaa,2023-12-01,1,Actual
145059,145059,c427e6da-b8a0-4556-8696-15e260cb846c,NaN,A3C74456P,2023-12-01,100.62,ERROR,Bank account information not available,2023-12-01,12,2023,NEW,SIN_CLABE,c427e6da-b8a0-4556-8696-15e260cb846c,2023-12-01,1,Actual
145060,145060,13167626-0e0a-49a9-8505-96de19ff2b6c,NaN,AGBSK5F2H,2023-12-01,4.79,ERROR,Bank account information not available,2023-12-01,12,2023,NEW,SIN_CLABE,13167626-0e0a-49a9-8505-96de19ff2b6c,2023-12-01,1,Actual
145061,145061,9ef4ae96-49ee-4beb-9f0c-c941992f5174,NaN,A6L28XYC3,2023-12-01,901.71,ERROR,Bank account information not available,2023-12-01,12,2023,NEW,SIN_CLABE,9ef4ae96-49ee-4beb-9f0c-c941992f5174,2023-12-01,1,Actual
145062,145062,e99ce28f-e3eb-425f-b734-4165a13aac74,NaN,AFZNM823P,2023-12-01,9.58,ERROR,Bank account information not available,2023-12-01,12,2023,NEW,SIN_CLABE,e99ce28f-e3eb-425f-b734-4165a13aac74,2023-12-01,1,Actual


In [10]:
query_error_current_month.shape

(11085, 17)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# AGRUPADO POR ESQUEMA

In [14]:
group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_7608\3966531006.py:1: FutureWarning: The provided callable <function sum at 0x000001B9F584F310> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_7608\3966531006.py:1: FutureWarning: The provided callable <function sum at 0x000001B9F584F310> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_7608\3966531006.py:1: FutureWarning: The provided callable <function sum at 0x000001B9F584F310> is currently using

ultimo_mov_date_only      amount                       \
esquema_query                      ERROR_USUAL            SIN_CLABE   
status_cubeta                            ERROR        NEW     ERROR   
0                       2023-12-01       24.00 137,439.56      0.00   
1                       2023-12-02   14,970.37  79,391.05      0.00   
2                       2023-12-03   14,406.13 215,822.40      0.00   
3                       2023-12-04   12,769.91 238,398.78      0.00   
4                       2023-12-05   15,734.57  83,676.17      0.00   

                                     
esquema_query                   All  
status_cubeta        NEW             
0              94,524.61 231,988.17  
1              90,857.43 185,218.85  
2             104,710.07 334,938.60  
3              83,972.70 335,141.39  
4              82,780.59 182,191.33

In [15]:
group_esquema.shape

(32, 6)

In [16]:
writer = pd.ExcelWriter('FASE VI. DATA POLIZA ERROR-SIN CLABE DICIEMBRE 2023.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
query_error_current_month.to_excel(writer, sheet_name='DETALLE',index=False)
writer.close()